# Modular approach

On this noteboook, we try to add and preprocess our own dataset manually and make our own config


https://elitedatascience.com/keras-tutorial-deep-learning-in-python

https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e

In [1]:
# imports
import matplotlib.pyplot as plt
from loguru import logger
import tensorflow as tf

import argparse
import contextlib
import os
import shutil
import sys

import main
import constants
import contributivity
import multi_partner_learning
import scenario
import utils
from dataset import Dataset

from pathlib import Path


import numpy as np
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

Using TensorFlow backend.


# Create custom scenario from mandatory parametters

In [2]:
scenario_params = {
    'partners_count': 3,
    'epoch_count': 2,                            # Not mandatory
    'minibatch_count': 2,                        # Not mandatory
    'amounts_per_partner': [0.2, 0.5, 0.3],
    'samples_split_option': ['basic', 'random']
}

Every other parametter will be set to its default value

In [3]:
experiment_path = Path(r"C:\GitHub\distributed-learning-contributivity\experiments\trash")

In [4]:
current_scenario = scenario.Scenario(
        scenario_params,
        experiment_path
    )

2020-08-04 15:28:22.337 | DEBUG    | scenario:__init__:52 - Dataset selected: mnist
2020-08-04 15:28:22.338 | DEBUG    | scenario:__init__:87 - Computation use the full dataset for scenario #1
2020-08-04 15:28:22.409 | INFO     | scenario:__init__:279 - ### Description of data scenario configured:
2020-08-04 15:28:22.410 | INFO     | scenario:__init__:280 -    Number of partners defined: 3
2020-08-04 15:28:22.410 | INFO     | scenario:__init__:281 -    Data distribution scenario chosen: random
2020-08-04 15:28:22.411 | INFO     | scenario:__init__:282 -    Multi-partner learning approach: fedavg
2020-08-04 15:28:22.411 | INFO     | scenario:__init__:283 -    Weighting option: uniform
2020-08-04 15:28:22.412 | INFO     | scenario:__init__:284 -    Iterations parameters: 2 epochs > 2 mini-batches > 8 gradient updates per pass
2020-08-04 15:28:22.412 | INFO     | scenario:__init__:290 - ### Data loaded: mnist
2020-08-04 15:28:22.413 | INFO     | scenario:__init__:291 -    48000 train data

# Create Data Set

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(X_train.shape[0],  28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

input_shape = (28, 28, 1)
num_classes = 10

# Create Preprocessing function

In [6]:
def preprocess_dataset_labels(y):
    y = np_utils.to_categorical(y, 10)
    return y

# Create Model

In [7]:
def generate_new_model_for_dataset():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Assignate dataset to scenario

In [8]:
current_scenario.dataset = Dataset(
    "my_dataset",
    X_train,
    X_test,
    y_train,
    y_test,
    input_shape,
    num_classes,
    preprocess_dataset_labels,
    generate_new_model_for_dataset
)

In [9]:
# Check Scenario name
print(current_scenario.dataset.name)

my_dataset


# Split train and validation sets

In [10]:
current_scenario.dataset.train_val_split()

# Legacy 

In [11]:
current_scenario.partners_list = []

# Run scenario

In [12]:
main.run_scenario(current_scenario)

2020-08-04 15:28:23.041 | INFO     | scenario:split_data:534 - ### Splitting data among partners:
2020-08-04 15:28:23.042 | INFO     | scenario:split_data:535 -    Simple split performed.
2020-08-04 15:28:23.042 | INFO     | scenario:split_data:536 -    Nb of samples split amongst partners: 38880
2020-08-04 15:28:23.043 | INFO     | scenario:split_data:538 -    Partner #0: 7776 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-04 15:28:23.043 | INFO     | scenario:split_data:538 -    Partner #1: 19440 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-04 15:28:23.044 | INFO     | scenario:split_data:538 -    Partner #2: 11664 samples with labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2020-08-04 15:28:23.177 | DEBUG    | scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #0: 486
2020-08-04 15:28:23.178 | DEBUG    | scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #1: 1215
2020-08-04 15:28:23.178 | DEBUG    | scenario:compute_batch_sizes:582 -  

0

# Results

In [13]:
df_results = current_scenario.to_dataframe()
df_results["random_state"] = 1
df_results["scenario_id"] = 1
print(df_results.columns)

Index(['aggregation_weighting', 'dataset_fraction_per_partner', 'dataset_name',
       'epoch_count', 'final_relative_nb_samples',
       'gradient_updates_per_pass_count', 'is_early_stopping',
       'learning_computation_time_sec', 'minibatch_count',
       'mpl_nb_epochs_done', 'mpl_test_score',
       'multi_partner_learning_approach', 'nb_samples_used', 'partners_count',
       'samples_split_description', 'scenario_name', 'short_scenario_name',
       'test_data_samples_count', 'train_data_samples_count', 'random_state',
       'scenario_id'],
      dtype='object')


In [14]:
print(df_results.mpl_test_score)

0    0.807
Name: mpl_test_score, dtype: float64
